In [1]:
import numpy as np
import pandas as pd

import scaper, os

In [2]:
cwd = os.getcwd()

print("Dir: ", cwd)

# change the current path to the data root
os.chdir(os.path.join(cwd, "data"))
print("Dir: ", os.getcwd())

cwd = os.getcwd()

Dir:  /scratch/IOSZ/waveformer/multimod-sound-separation/multimod-waveformer
Dir:  /scratch/IOSZ/waveformer/multimod-sound-separation/multimod-waveformer/data


# Scaper jam generator

#### For training & val datasets: 'TAU-urban-acoustic-scenes-2019-development'
#### For testing dataset: 'TAU-urban-acoustic-scenes-2019-evaluation'

#### Train/val/test: 10000/200/300  // initial

#### Train/val/test: 15000/1000/2000 // current

### Parameters & locations

In [20]:
DATA_SPLIT = 'test'  # define data split

n_soundscapes = 2000  # number of .jams files to generate
duration = 6.0  # duration of the soundscape in seconds
ref_db = -30 # reference dB for the soundscape
min_events = 1  # minimum number of events per soundscape
max_events = 1  # maximum number of events per soundscape
max_event_duration = 5.0  # maximum possible duration of a foreground event
fg_snr_low = 5
fg_snr_high = 10

no_audio = True  # if the jam process should include the raw audio mix as well
allow_repeated_label = False  # if you can have multiple events of the same sound label
allow_repeated_source = False  # if you can have duplicates of the source files

seed = 42  # NOT USED; turns out it always selects the same file if used

print("Ref DB: ", ref_db, '; SNR: ',fg_snr_low, "-", fg_snr_high)#, "SEED: ", seed)
print("No audio out: ", no_audio)

Ref DB:  -30 ; SNR:  5 - 10
No audio out:  True


In [21]:
# Define constant parameters for the soundscape
fg_folder = os.path.join('CVSoundScapes', 'CV-13-mini', DATA_SPLIT)  # foreground folder

if DATA_SPLIT == 'train' or DATA_SPLIT == 'val' :
    bg_folder = os.path.join('CVSoundScapes', 'TAU-acoustic-sounds', 
                             'TAU-urban-acoustic-scenes-2019-development')
else:
    bg_folder = os.path.join('CVSoundScapes', 'TAU-acoustic-sounds', 
                             'TAU-urban-acoustic-scenes-2019-evaluation')
    
print('Fg folder: ', fg_folder)
print('Bg folder: ', bg_folder)
    
out_folder = os.path.join('CVSoundScapes', 'jams', DATA_SPLIT)

Fg folder:  CVSoundScapes/CV-13-mini/test
Bg folder:  CVSoundScapes/TAU-acoustic-sounds/TAU-urban-acoustic-scenes-2019-evaluation


### Generate the jams

In [22]:
#df = pd.read_csv(os.path.join('data','CVSoundScapes', 'CV-13-mini', 'train.csv'))

In [23]:
# For each soundscape
for n in range(n_soundscapes):

    # Create a Scaper instance
    sc = scaper.Scaper(duration, fg_folder, bg_folder)
    sc.protected_labels = []
    sc.ref_db = ref_db

    # Add background
    sc.add_background(
        label=('const', 'audio'),
        source_file=('choose', []),
        source_time=('const', 0))

    # Add random number of foreground events
    num_events = np.random.randint(min_events, max_events+1)
    
    for _ in range(num_events):
        sc.add_event(
            label=('choose', []),  
            source_file=('choose', []),  
            source_time=('const', 0),
            event_time=('uniform', 0, duration - max_event_duration),
            event_duration=('const', max_event_duration), 
            snr=('uniform', fg_snr_low, fg_snr_high),  
            pitch_shift=None,
            time_stretch=None)    
            
    # mkdir for this sample
    os.mkdir(os.path.join(cwd, out_folder, str(n)))
    
    # Generate the .jams file
    audio_file = os.path.join(out_folder, str(n), f'mixture.wav')
    jams_file = os.path.join(out_folder, str(n), f'mixture.jams')
    txt_file = os.path.join(out_folder, str(n), f'mixture.txt')
    
    sc.generate(audio_file, jams_file, allow_repeated_label=allow_repeated_label, allow_repeated_source=allow_repeated_source,
                reverb=0, disable_sox_warnings=True, no_audio=no_audio, txt_path=txt_file)
    
    # optional: create a csv version of the txt file
    df = pd.read_csv(os.path.join(out_folder, str(n), 'mixture.txt'), sep='\t', header=None)
    df.columns = ['start', 'end', 'label']  # the expected columns
    
    df.to_csv(os.path.join(out_folder, str(n), 'gt_events.csv'), index=False, header=True, sep='\t')

/home/iosz/.conda/envs/waveformer/lib/python3.8/site-packages/scaper/core.py:1501: ScaperWarning: female event duration (5.00) is greater that source duration (4.43), changing to 4.43
  warnings.warn(
/home/iosz/.conda/envs/waveformer/lib/python3.8/site-packages/scaper/core.py:1501: ScaperWarning: female event duration (5.00) is greater that source duration (4.25), changing to 4.25
  warnings.warn(
/home/iosz/.conda/envs/waveformer/lib/python3.8/site-packages/scaper/core.py:1501: ScaperWarning: male event duration (5.00) is greater that source duration (4.54), changing to 4.54
  warnings.warn(
/home/iosz/.conda/envs/waveformer/lib/python3.8/site-packages/scaper/core.py:1501: ScaperWarning: female event duration (5.00) is greater that source duration (4.72), changing to 4.72
  warnings.warn(
/home/iosz/.conda/envs/waveformer/lib/python3.8/site-packages/scaper/core.py:1501: ScaperWarning: female event duration (5.00) is greater that source duration (2.70), changing to 2.70
  warnings.war